In [1]:
pip install torch transformers langchain gradio sacremoses

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = "/kaggle/input/elgpt-advance/final_t5-base_medical_chatbot"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

2025-05-17 21:58:10.687469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747519090.710352     129 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747519090.717500     129 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from datasets import Dataset
import torch
import numpy as np

file_path = "ai-medical-chatbot.csv"
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "yousefsaeedian/ai-medical-chatbot",
    file_path,
)

df["input_text"] = df.apply(lambda row: f"### Instruction:\n{row['Patient']}", axis=1)
df["target_text"] = df.apply(lambda row: f"{row['Doctor']}", axis=1)
df = df[40000:42000]

dataset = Dataset.from_pandas(df[["input_text", "target_text"]])

/tmp/ipykernel_129/1258301978.py:20: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [5]:
!pip install -q evaluate rouge_score

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)

Using device: cuda


In [ ]:
import evaluate
from tqdm import tqdm
import numpy as np

max_model_length = 1024
max_new_tokens = 128
max_input_length = max_model_length - max_new_tokens

predictions = []
references = []
for example in tqdm(dataset.select(range(2000))):
    input_text = example["input_text"]
    reference_text = example["target_text"]

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_input_length,
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            repetition_penalty=1.2,
        )

    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = pred_text.split("Doctor:")[-1].strip()
    predictions.append(response)
    references.append(reference_text)

100%|██████████| 2000/2000 [41:56<00:00,  1.26s/it]


In [17]:

rouge_metric = evaluate.load("rouge")   
bleu_metric = evaluate.load("bleu") 
meteor_metric = evaluate.load("meteor") 
bleu_metric.add_batch(
    predictions=predictions,   
    references=[[ref] for ref in references]  
)
meteor_metric.add_batch(predictions=predictions, references=references)
rouge_metric.add_batch(predictions=predictions, references=references)


rouge_results = rouge_metric.compute()
bleu_result = bleu_metric.compute()
meteor_result = meteor_metric.compute()

print("ROUGE Scores:")
for k, v in rouge_results.items():
    print(f"{k}: {v:.4f}")

print(f"\nBLEU Score: {bleu_result['bleu']:.4f}")
print(f"METEOR Score: {meteor_result['meteor']:.4f}")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ROUGE Scores:
rouge1: 0.1310
rouge2: 0.0171
rougeL: 0.0825
rougeLsum: 0.0825

BLEU Score: 0.0072
METEOR Score: 0.0852
